# read_tmdb_data.ipynb

This notebook reads JSON data from TMDB's API and stores it as CSV files:
- **movies.csv**: Data on approx. 27000 movies from the United States from 2000-2023.
- **persons.csv**: Data on the directors and top 10 cast members in these movies.

In [1]:
import os
import requests
from dotenv import load_dotenv
from time import sleep
import pandas as pd

load_dotenv()
tmdb_api_token = os.getenv("TMDB_API_TOKEN")

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {tmdb_api_token}"
}

REQUEST_DELAY_SECONDS = 0.02

First we request movie_ids for all movies that live up to these requirements:
- From the United States
- From the years 2000-2023
- With original_langauge = english
- With TMDB vote count ≥ 10

In [2]:
base_url = (
    "https://api.themoviedb.org/3/discover/movie"
    "?include_adult=false"
    "&include_video=false"
    "&with_origin_country=US"
    "&with_original_language=en"
    "&vote_count.gte=10"
    "&sort_by=primary_release_date.asc"
)

movie_ids = []

# Loop through the years
for year in range(2000, 2024):

    # Loop through the pages (TMDB uses a maximum of 500 pages)
    for page in range(1, 501):
        url = f"{base_url}&primary_release_year={year}&page={page}"
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            raise Exception(f"year: {year} page: {page} status_code: {response.status_code} text: {response.text}")
    
        # Extract movie_results from the response
        movie_results = response.json().get("results") 
    
        # Stop if we have reached the last page
        if not movie_results:
            break

        # Extend movie_ids list with the ones from movie_results
        movie_ids.extend([movie["id"] for movie in movie_results])
    
        sleep(REQUEST_DELAY_SECONDS)

print(f"Number of movie ids found: {len(movie_ids)}")

Number of movie ids found: 27294


Then we request movie data for all these movie_ids, and store that in a dataframe and a CSV file

In [3]:
all_movies = []

# Loop through all movie_ids and request movie data
for movie_id in movie_ids:
    url = f"https://api.themoviedb.org/3/movie/{movie_id}?append_to_response=credits"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"movie_id: {movie_id} status_code: {response.status_code} text: {response.text}")
    
    # Extract the movie data from the response as a dictionary
    movie = response.json()

    # Find the directors and put director_person_ids in the dictionary
    movie["director_person_ids"] = []
    for credit in movie["credits"]["crew"]:
        if(credit["job"]=="Director"):
            movie["director_person_ids"].append(credit["id"])

    # Simplify child dictionaries to list of ids
    movie["genre_ids"] = [genre["id"] for genre in movie["genres"]]
    movie["spoken_languages"] = [language["iso_639_1"] for language in movie["spoken_languages"]]
    movie["production_company_ids"] = [company["id"] for company in movie["production_companies"]]
    movie["production_countries"] = [country["iso_3166_1"] for country in movie["production_countries"]]
    movie["collection_id"] = movie["belongs_to_collection"]["id"] if movie["belongs_to_collection"] else pd.NA
    movie["cast_person_ids"] = [cast_member["id"] for cast_member in movie["credits"]["cast"]]
    movie["cast_credit_ids"] = [cast_member["credit_id"] for cast_member in movie["credits"]["cast"]]
    movie["crew_person_ids"] = [crew_member["id"] for crew_member in movie["credits"]["crew"]]
    movie["crew_credit_ids"] = [crew_member["credit_id"] for crew_member in movie["credits"]["crew"]]
    del movie['genres']
    del movie['production_companies']
    del movie["belongs_to_collection"]   
    del movie["credits"]
    
    all_movies.append(movie)
    sleep(REQUEST_DELAY_SECONDS)  

df_movies = pd.DataFrame(all_movies)
df_movies.rename(columns={"id": "movie_id"}, inplace=True)
df_movies.to_csv("../movie_data/movies.csv", index=False)
print(f"Number of movies collected: {len(df_movies)}")
df_movies

Number of movies collected: 27294


,adult,backdrop_path,budget,homepage,movie_id,imdb_id,origin_country,original_language,original_title,overview,...,vote_average,vote_count,director_person_ids,genre_ids,production_company_ids,collection_id,cast_person_ids,cast_credit_ids,crew_person_ids,crew_credit_ids
0,False,/dD90r6NQ8cFgYjjYGSLRQLCdJWN.jpg,0,,515728,tt0191181,[US],en,Hitch,Two friends are on a road trip and a one-sided...,...,4.900,10,[131388],[18],[],<NA>,"[1230580, 2030046]","[5ac28e640e0a260c140239f0, 5ae16159c3a36876ab0...","[1434896, 131388, 131388, 131388, 131388, 1360...","[6454f309c044290143e43376, 6454f31187a27a011b1..."
1,False,/ifq88qw3vgoKlUyw0OAmPQCSqBc.jpg,0,,300236,tt0259233,[US],en,Carnage: The Legend of Quiltface,Four students set out for the barren Nevada de...,...,2.500,10,[103123],[27],"[4708, 110668]",<NA>,"[98740, 99106, 98276, 1685427, 1771744, 177174...","[58bf16e6925141608406b270, 6251f3b9a6c104320b2...","[103123, 98868, 103123, 1001648, 103123, 10016...","[62c5bceaf794ad00bf5a8867, 62c5bce19ba86a00ee4..."
2,False,/c5UlEYHM2xuSrfxESg1gZSpVAEB.jpg,0,,96716,tt0128977,[US],en,The Bumblebee Flies Anyway,An amnesiac youth tries to piece together his ...,...,6.200,30,[126537],"[18, 10749]","[1596, 1363]",<NA>,"[109, 21197, 16407, 1223778, 38581, 303197, 56...","[52fe49be9251416c750d1f8f, 52fe49be9251416c750...","[1534680, 2556479, 1516275, 68126, 1516278, 27...","[60c1e4cb39a45d0040c5cd4b, 60c1e503960cde00562..."
3,False,None,0,,71618,tt0198284,[US],en,After Sex,A group of attractive women get together for a...,...,5.000,27,[176312],"[35, 18, 10749]","[85165, 86531]",<NA>,"[170638, 12519, 15110, 3208, 51670, 61962, 117...","[53d9db080e0a2652f0001583, 52fe483ec3a368484e0...","[176312, 1470931, 33008, 17210, 17211, 954441,...","[52fe483ec3a368484e0ef37f, 5564e59bc3a368740e0..."
4,False,/nKD4M8Oyuh5aE9EBR29A9WnJKxE.jpg,0,,66131,tt0346794,[US],en,A Constant Forge,"One of the great mavericks of cinema, John Cas...",...,6.600,13,[544690],[99],[],<NA>,"[11147, 5950, 10556, 10127, 856, 2314, 1629458...","[5a9dfc9e0e0a2671fb009de8, 5a9dfc11c3a36842820...","[544690, 3399184, 544690, 961119, 3399183, 339...","[52fe472bc3a368484e0b89d9, 61f15d7dcd204600bc7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27289,False,/gub9e0qRZWmmzTV77A6T95HELIF.jpg,40000000,https://www.mgm.com/movies/the-boys-in-the-boat,823452,tt1856080,[US],en,The Boys in the Boat,The triumphant underdog story of the Universit...,...,7.233,311,[1461],"[18, 36]","[1782, 143790, 210099]",<NA>,"[33192, 1371041, 27172, 1522758, 1780950, 2725...","[6205b7c5ce5d8200c83376f3, 61c21146904f6d00406...","[3069889, 1461, 17148, 1500871, 1578875, 18195...","[6088693d84448e007932a40c, 6088695754a09800294..."
27290,False,/rSle4ix2IZ6UMgCDY8L3ngdOmSZ.jpg,90000000,https://www.warnerbros.com/movies/color-purple,558915,tt1200263,[US],en,The Color Purple,A decades-spanning tale of love and resilience...,...,7.049,326,[1507725],[18],"[174, 3298, 56, 150841, 200643, 216687]",<NA>,"[165909, 40036, 1075037, 91671, 1154054, 34767...","[61fc8db841429100a2189044, 61f966bfeee1860044e...","[3511353, 4445624, 4445622, 73350, 2167926, 21...","[62b89b15a61de103b61eed9b, 65876f514772155add4..."
27291,False,/vhTsS3K79lsyZNZ7t33IQiSREsO.jpg,0,https://www.netflix.com/title/81449757,1215278,tt30225680,"[US, GB]",en,Hell Camp: Teen Nightmare,Out-of-control teens across America were sent ...,...,5.663,46,[2283876],[99],[26359],<NA>,"[5332322, 38406]","[67e5d2703e65c8ea88ba1e0f, 67e60712421eb8c331b...","[2283876, 3248862, 2060153, 4208975, 1414935, ...","[65734ffc1c635b00c3a9bfbe, 67e5d17f421eb8c331b..."
27292,False,/zS1OiCw3opLQYxjmhqhOQ7D7YyD.jpg,0,,1156189,tt28490873,[US],en,Ryuichi Sakamoto: Opus,"""Ars longa, vita brevis"" – art is long, life i...",...,7.861,18,[2010321],"[10402, 99]","[188205, 11561, 4667, 46298]",<NA>,[11382],[64bfde5d8c0a4800aeefb198],"[3056, 1610252, 2247870, 3406563, 4240016, 152...","[64ec9e951feac100fe5e313e, 64ec9e335258ae014df..."


Now we find person ids for the directors in each of the movies.

And then we find person ids for the top 10 cast members in each of the movies.

And we combine these two lists to a unique_person_ids set.

In [4]:
director_person_ids = df_movies['director_person_ids'].explode().dropna().tolist()

top_10_cast_person_ids = (
    df_movies['cast_person_ids']
    .apply(lambda ids: ids[:10])   # get first 10 for each movie
    .explode()                     # flatten into one Series
    .dropna()
    .tolist()                      # convert to plain list
)

unique_person_ids = set(director_person_ids) | set(top_10_cast_person_ids)
len(unique_person_ids)

104929

Finally we request person data for all these unique_person_ids, and store that in a dataframe and a CSV file.

In [5]:
all_persons = []

for person_id in unique_person_ids:
    url = f"https://api.themoviedb.org/3/person/{person_id}"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"person_id: {person_id} status_code: {response.status_code} text: {response.text}")
    
    # Extract the person data from the response
    person = response.json()
    
    all_persons.append(person)
    sleep(REQUEST_DELAY_SECONDS)
    
    
df_persons = pd.DataFrame(all_persons)
df_persons.rename(columns={"id": "person_id"}, inplace=True)
df_persons.to_csv("../movie_data/persons.csv", index=False)
print(f"Number of persons collected: {len(df_persons)}")
df_persons

ReadTimeout: HTTPSConnectionPool(host='api.themoviedb.org', port=443): Read timed out. (read timeout=None)